In [10]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("cuda is available")
else:
    print("cuda is NOT available")

import numpy as np
from tqdm import tqdm
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import time
import copy
from moving_average import moving_average_1d

import importlib
import policy
importlib.reload(policy)
from policy import PolicyNN

from nn_functions import surrogate

import sys
sys.path.append('../1_model')
from TiDE import TideModule, quantile_loss  


cuda is available


In [11]:
import torch
import pickle

# Load model
with open('TiDE_params_single_track_square_MV_temp_depth_less_cov_0915_w50_p50.pkl', 'rb') as file:
    nominal_params = pickle.load(file)

TiDE = nominal_params['model'].to(device)
total_params = sum(p.numel() for p in TiDE.parameters())

In [12]:
device = torch.device("cuda:0")

# ── Constants ─────────────────────────────────────
INPUT_DATA_DIR = "data"
SIM_DIR_NAME = "single_track_square"
BASE_LASER_FILE_DIR = "laser_power_profiles/csv"
CLOUD_TARGET_BASE_PATH = "result"
solidus_temp = 1600
window = 50
sim_interval = 5
init_runs = 50
P = 50

model_path = "/home/ftk3187/github/DPC_research/02_DED/4_policy_0725/trainresults/policy_model_discreteshift_3L_1024H_s1_c0_case19.pth"

# ── Load model ─────────────────────────────────────
model = PolicyNN(
    past_input_dim=6,
    future_input_dim=6,
    output_dim=1,
    p=P,
    window=window,
    hidden_dim=1024,
    n_layers=3,
    dropout_p=0.1
).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

PolicyNN(
  (input_layer): Linear(in_features=600, out_features=1024, bias=True)
  (input_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (norm_layers): ModuleList(
    (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (output_layer): Linear(in_features=1024, out_features=50, bias=True)
)

In [13]:
from typing import Optional, Tuple
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from pickle import dump
from sklearn.preprocessing import MinMaxScaler
import os

import shutil
import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

from GAMMA_obj_temp_depth import GAMMA_obj

import importlib
import policy
importlib.reload(policy)
from policy import PolicyNN

In [14]:
# values from user
x_min = torch.tensor([[0.0, 0.75, 0.75, 504.26]], dtype=torch.float32).to(device)
x_max = torch.tensor([[7.5, 20.0, 20.0, 732.298]], dtype=torch.float32).to(device)

y_min = torch.tensor([[436.608, -0.559]], dtype=torch.float32).to(device)
y_max = torch.tensor([[4509.855, 0.551]], dtype=torch.float32).to(device)


def normalize_x(x, dim_id):
    x_min_selected = x_min[0, dim_id]
    x_max_selected = x_max[0, dim_id]
    return 2 * (x - x_min_selected) / (x_max_selected - x_min_selected) - 1

def inverse_normalize_x(x_norm, dim_id):
    x_min_selected = x_min[0, dim_id]
    x_max_selected = x_max[0, dim_id]
    return 0.5 * (x_norm + 1) * (x_max_selected - x_min_selected) + x_min_selected

def normalize_y(y, dim_id):
    y_min_selected = y_min[0, dim_id]
    y_max_selected = y_max[0, dim_id]
    return 2 * (y - y_min_selected) / (y_max_selected - y_min_selected) - 1

def inverse_normalize_y(y_norm, dim_id):
    y_min_selected = y_min[0, dim_id]
    y_max_selected = y_max[0, dim_id]
    return 0.5 * (y_norm + 1) * (y_max_selected - y_min_selected) + y_min_selected


In [15]:
import copy
import torch

def simulate_policy_horizon_for_visualization(GAMMA_obj, policy_model, TiDE, P, window):
    """
    Returns:
        - tide_temp_pred: [50]
        - tide_depth_pred: [50]
        - gamma_temp_sim: [50]
        - gamma_depth_sim: [50]
    """

    # ─────────────────────────────────────────────────────────────
    # 1. Policy Input 생성 (정책 모델용 → normalized)
    # ─────────────────────────────────────────────────────────────
    mp_temp_ref = GAMMA_obj.ref[GAMMA_obj.MPC_counter:GAMMA_obj.MPC_counter + P]
    mp_temp_ref_t = torch.tensor(mp_temp_ref, dtype=torch.float32, device=device).reshape(1, P, 1)

    mp_temp_past_t = GAMMA_obj.x_past.T.unsqueeze(0).to(device)  # [1, 50, 2]
    laser_past_t = GAMMA_obj.u_past.view(1, -1, 1).to(device)     # [1, 50, 1]

    fix_cov_past = GAMMA_obj.fix_cov_all[GAMMA_obj.MPC_counter - window:GAMMA_obj.MPC_counter, :]
    fix_cov_future = GAMMA_obj.fix_cov_all[GAMMA_obj.MPC_counter:GAMMA_obj.MPC_counter + P, :]

    fix_cov_past_t = torch.tensor(fix_cov_past, dtype=torch.float32, device=device).unsqueeze(0)  # [1, 50, 3]
    fix_cov_future_t = torch.tensor(fix_cov_future, dtype=torch.float32, device=device).unsqueeze(0)  # [1, 50, 3]

    # Normalize for policy model
    fix_cov_past_s = normalize_x(fix_cov_past_t, dim_id=[0, 1, 2])
    fix_cov_future_s = normalize_x(fix_cov_future_t, dim_id=[0, 1, 2])
    laser_past_s = normalize_x(laser_past_t, dim_id=[3])
    mp_temp_past_s = normalize_y(mp_temp_past_t, dim_id=[0, 1])
    mp_temp_ref_s = normalize_y(mp_temp_ref_t, dim_id=[0])[:, :, 0].unsqueeze(-1)

    depth_lower_const = 0.1423
    depth_upper_const = 0.4126
    y_const_s = torch.tensor([[depth_lower_const, depth_upper_const]] * P,
                             dtype=torch.float32, device=device).reshape(1, P, 2)

    policy_in_past = torch.cat((fix_cov_past_s, laser_past_s, mp_temp_past_s), dim=2)  # [1, 50, 6]
    policy_in_future = torch.cat((fix_cov_future_s, mp_temp_ref_s, y_const_s), dim=2)  # [1, 50, 6]

    # ─────────────────────────────────────────────────────────────
    # 2. Policy inference → u_pred (normalized) & u_seq (denorm)
    # ─────────────────────────────────────────────────────────────
    with torch.no_grad():
        u_pred = policy_model((policy_in_past, policy_in_future))  # [1, 50, 1]
        u_seq_norm = u_pred[0]  # [50, 1]
        u_seq = inverse_normalize_x(u_seq_norm, dim_id=[3])  # [50, 1]
        

    # ─────────────────────────────────────────────────────────────
    # 3. TiDE 예측 수행 (비정책 입력 형식: unnormalized + reordered)
    # ─────────────────────────────────────────────────────────────
    # TiDE past_cov: [y_past, x_past] → [1, 50, 6]
    past_cov = torch.cat((mp_temp_past_s, fix_cov_past_s), dim=2)

    # TiDE future_cov: [x_future, u_pred] → [1, 50, 4]
    future_cov = torch.cat((fix_cov_future_s, u_pred.to(device)), dim=2)

    with torch.no_grad():
        tide_out = TiDE((past_cov, future_cov, None))  # [1, 50, quantile, 2]
        tide_out_med = tide_out[:, :, :, 1].squeeze(0).cpu().numpy()  # [50, 2]
        tide_temp_pred = inverse_normalize_y(torch.tensor(tide_out_med[:, 0], device=device), dim_id=[0])
        tide_depth_pred = inverse_normalize_y(torch.tensor(tide_out_med[:, 1], device=device), dim_id=[1])


   # ─────────────────────────────────────────────────────────────
    # ── 4. GAMMA 시뮬레이션 (cumulative update) ────────────
    # ─────────────────────────────────────────────────────────────
    gamma_temp_sim = []
    gamma_depth_sim = []

    gamma_copy = copy.deepcopy(GAMMA_obj)

    # ✅ 명시적 초기화 (GAMMA 내부 상태 재설정)
    gamma_copy.x_sys_current = gamma_copy.x_past[:, -1].reshape(2, 1)
    gamma_copy.x_hat_current = gamma_copy.x_sys_current.flatten()
    gamma_copy.MPC_counter = window
    gamma_copy.x_past_save = gamma_copy.x_past.T.clone()
    gamma_copy.u_past_save = gamma_copy.u_past.view(-1, 1).clone()

    for i in range(P):
        u_val = float(u_seq[i].item())

        # 한 스텝 시뮬레이션 실행
        x_t, d_t = gamma_copy.run_sim_interval(u_val)


        # 출력 저장
        gamma_temp_sim.append(x_t)
        gamma_depth_sim.append(d_t)

        # 시계열 업데이트
        gamma_copy.x_past[:, :-1] = gamma_copy.x_past[:, 1:]
        gamma_copy.x_past[0, -1] = x_t
        gamma_copy.x_past[1, -1] = d_t

        gamma_copy.u_past[:-1] = gamma_copy.u_past[1:].clone()
        gamma_copy.u_past[-1] = u_val

        # 상태 변수 업데이트
        gamma_copy.x_hat_current = torch.tensor([x_t, d_t], device=device)
        gamma_copy.x_sys_current = torch.tensor([[x_t], [d_t]], device=device)
        gamma_copy.MPC_counter += 1

        # 전체 시계열 기록도 업데이트 (plot 대비)
        new_state = torch.tensor([[x_t, d_t]], device=gamma_copy.x_past_save.device)
        gamma_copy.x_past_save = torch.cat((gamma_copy.x_past_save, new_state), dim=0)

        new_u = torch.tensor([[u_val]], device=gamma_copy.u_past_save.device)
        gamma_copy.u_past_save = torch.cat((gamma_copy.u_past_save, new_u), dim=0)


    return (
        tide_temp_pred.cpu().numpy(),   # [50]
        tide_depth_pred.cpu().numpy(),  # [50]
        np.array(gamma_temp_sim),       # [50]
        np.array(gamma_depth_sim),      # [50]
        u_seq.cpu().numpy()             # [50, 1]
    )



In [16]:
import matplotlib.pyplot as plt
import numpy as np

def plot_tide_vs_gamma_stepwise(
    step_idx,
    GAMMA_obj,
    tide_temp,
    tide_depth,
    gamma_temp,
    gamma_depth,
    u_seq,
    P=50
):
    time_steps = np.arange(step_idx, step_idx + P)

    fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)

    # ── Temperature Plot ──────────────────────────────
    axs[0].plot(time_steps, tide_temp, label="TiDE Temp", linestyle='--', marker='o')
    axs[0].plot(time_steps, gamma_temp, label="GAMMA Temp", linestyle='-', marker='x')
    axs[0].set_ylabel("Melt Pool Temp (K)")
    axs[0].legend()
    axs[0].grid(True)

    # ── Depth Plot ────────────────────────────────────
    axs[1].plot(time_steps, tide_depth, label="TiDE Depth", linestyle='--', marker='o')
    axs[1].plot(time_steps, gamma_depth, label="GAMMA Depth", linestyle='-', marker='x')
    axs[1].set_ylabel("Melt Pool Depth (mm)")
    axs[1].legend()
    axs[1].grid(True)

    # ── Laser Power Input ─────────────────────────────
    axs[2].plot(time_steps, u_seq.squeeze(), label="Policy Laser Power", color='green', marker='.')
    axs[2].set_ylabel("Laser Power")
    axs[2].set_xlabel("Timestep")
    axs[2].legend()
    axs[2].grid(True)

    fig.suptitle(f"Step {step_idx} → {step_idx + P - 1}: TiDE vs GAMMA", fontsize=14)
    plt.tight_layout()
    plt.show()


In [17]:
import copy

# 레이저 번호 설정
laser_power_number = 15

# 원본 데이터 불러오기
csv_path = f"/home/ftk3187/github/DPC_research/02_DED/4_policy_0725/split_by_laser_power_number/laser_power_number_{laser_power_number}.csv"
df = pd.read_csv(csv_path)

# 고정 공변량
loc_Z  = df["Z"].to_numpy().reshape(-1, 1)
dist_X = df["Dist_to_nearest_X"].to_numpy().reshape(-1, 1)
dist_Y = df["Dist_to_nearest_Y"].to_numpy().reshape(-1, 1)
fix_covariates = torch.tensor(np.concatenate((loc_Z, dist_X, dist_Y), axis=1), dtype=torch.float32, device=device)

# 레이저 파워 시퀀스
laser_power_ref = torch.tensor(df["Laser_power"].to_numpy().reshape(-1, 1), dtype=torch.float32, device=device)

# ⛳ melt pool temperature → smoothing → tensor
mp_temp_raw = df["melt_pool_temperature"].to_numpy()
mp_temp_smooth = copy.deepcopy(mp_temp_raw)
mp_temp_smooth[1:-2] = (mp_temp_raw[:-3] + mp_temp_raw[1:-2] + mp_temp_raw[2:-1] + mp_temp_raw[3:]) / 4
mp_temp_ref = torch.tensor(mp_temp_smooth, dtype=torch.float32, device=device)


In [ ]:
max_steps = 20
GAMMA_obj = GAMMA_obj(
    INPUT_DATA_DIR="data",
    SIM_DIR_NAME="single_track_square",
    BASE_LASER_FILE_DIR="laser_power_profiles/csv",
    CLOUD_TARGET_BASE_PATH="result",
    solidus_temp=solidus_temp,
    window=window,
    init_runs=init_runs,
    sim_interval=sim_interval,
    laser_power_number=laser_power_number
)

# Step 1: 초기 상태 세팅
init_avg = torch.tensor(GAMMA_obj.run_initial_steps(), dtype=torch.float32, device=device)[:, -window:]
GAMMA_obj.ref = mp_temp_ref.clone()
GAMMA_obj.fix_cov_all = fix_covariates.clone()
GAMMA_obj.x_past = init_avg.clone()
GAMMA_obj.u_past = laser_power_ref[:window].clone()
GAMMA_obj.x_past_save = GAMMA_obj.x_past.T.clone()
GAMMA_obj.u_past_save = GAMMA_obj.u_past.view(-1, 1).clone()
GAMMA_obj.MPC_counter = window
GAMMA_obj.x_hat_current = GAMMA_obj.x_past[:, -1]
GAMMA_obj.x_sys_current = GAMMA_obj.x_hat_current.reshape(2, 1)

# Step 2: 시뮬레이션 루프
for step in range(1, max_steps + 1):
    # 정책 기반 예측 (50-step u_seq 예측)
    _, _, _, _, u_seq = simulate_policy_horizon_for_visualization(
        GAMMA_obj, policy_model=model, TiDE=TiDE, P=P, window=window
    )

    # u_seq[0] 만 적용 → 1 step 실행
    u_val = float(u_seq[0].item())
    x_t, d_t = GAMMA_obj.run_sim_interval(u_val)

    # 시계열 업데이트 (한 칸 shift)
    GAMMA_obj.x_past[:, :-1] = GAMMA_obj.x_past[:, 1:]
    GAMMA_obj.x_past[0, -1], GAMMA_obj.x_past[1, -1] = x_t, d_t
    GAMMA_obj.u_past[:-1] = GAMMA_obj.u_past[1:].clone()
    GAMMA_obj.u_past[-1] = u_val
    GAMMA_obj.x_hat_current = torch.tensor([x_t, d_t], device=device)
    GAMMA_obj.x_sys_current = GAMMA_obj.x_hat_current.reshape(2, 1)
    GAMMA_obj.MPC_counter += 1

    # Step 3: 마지막 step 에서만 전체 시각화 수행
    if step == max_steps:
        tide_temp, tide_depth, _, _, u_seq_final = simulate_policy_horizon_for_visualization(
            GAMMA_obj, policy_model=model, TiDE=TiDE, P=P, window=window
        )

        gamma_temp_sim, gamma_depth_sim = [], []
        for i in range(P):
            u_val = float(u_seq_final[i].item())
            x_t, d_t = GAMMA_obj.run_sim_interval(u_val)
            gamma_temp_sim.append(x_t)
            gamma_depth_sim.append(d_t)

            # 상태 업데이트
            GAMMA_obj.x_past[:, :-1] = GAMMA_obj.x_past[:, 1:]
            GAMMA_obj.x_past[0, -1], GAMMA_obj.x_past[1, -1] = x_t, d_t
            GAMMA_obj.u_past[:-1] = GAMMA_obj.u_past[1:].clone()
            GAMMA_obj.u_past[-1] = u_val
            GAMMA_obj.x_hat_current = torch.tensor([x_t, d_t], device=device)
            GAMMA_obj.x_sys_current = GAMMA_obj.x_hat_current.reshape(2, 1)
            GAMMA_obj.MPC_counter += 1

            if step % 5 == 0:
                print(f"[Step {step}] u: {u_val:.1f}, pred temp: {tide_temp[0]:.1f}, sim temp: {x_t:.1f}")

        # 시각화
        plot_tide_vs_gamma_stepwise(
            step_idx=GAMMA_obj.MPC_counter,
            GAMMA_obj=GAMMA_obj,
            tide_temp=tide_temp,
            tide_depth=tide_depth,
            gamma_temp=np.array(gamma_temp_sim),
            gamma_depth=np.array(gamma_depth_sim),
            u_seq=u_seq_final,
            P=P
        )


100%|██████████| 250/250 [00:05<00:00, 42.93it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:3 and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)